# PV Production Classification with Logistic Regression

Multi-class logistic regression model to classify PV production levels based on weather conditions.

## 1 - Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import copy

%matplotlib inline

## 2 - Load and Explore Data

In [ ]:
# Load data
data = pd.read_excel('Data/RawData.xlsx')

print(f"Dataset shape: {data.shape}")
print(f"\nColumns: {data.columns.tolist()}")
print(f"\nFirst rows:\n{data.head()}")

## 3 - Data Preprocessing

We classify PV production into 3 classes based on percentiles: Low (0-33%), Medium (33-66%), High (66-100%).

In [ ]:
# Filter out zero production values
data_filtered = data[data['electricity_kW'] > 0].copy()

print(f"Original dataset: {len(data)} samples")
print(f"After filtering zeros: {len(data_filtered)} samples")

# Extract features (X) and target (y)
X = data_filtered[['Temperature_celcius', 'Rain_mm_per_h', 'Snow_mm_per_h', 
          'Air density_kg_per_m3', 'Ground_level_solar_irradiance_W_per_m2', 
          'Cloud_cover_fraction', 'wind_speed_m_per_s']].values

y_continuous = data_filtered['electricity_kW'].values

# Create classes based on percentiles
percentile_33 = np.percentile(y_continuous, 33)
percentile_66 = np.percentile(y_continuous, 66)

print(f"\nLow production: 0 - {percentile_33:.2f} kW")
print(f"Medium production: {percentile_33:.2f} - {percentile_66:.2f} kW")
print(f"High production: {percentile_66:.2f}+ kW")

# Create class labels (0: Low, 1: Medium, 2: High)
y = np.zeros(len(y_continuous))
y[y_continuous > percentile_33] = 1
y[y_continuous > percentile_66] = 2

print(f"\nClass distribution:")
unique, counts = np.unique(y, return_counts=True)
for cls, cnt in zip(unique, counts):
    print(f"  Class {int(cls)}: {cnt} samples ({100*cnt/len(y):.1f}%)")

### Train/Test Split and Normalization

In [ ]:
# 80-20 random split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

# Transpose for neural network format (features in rows, samples in columns)
X_train_norm = X_train_norm.T
X_test_norm = X_test_norm.T
y_train = y_train.reshape(1, -1)
y_test = y_test.reshape(1, -1)

m_train = X_train_norm.shape[1]
m_test = X_test_norm.shape[1]

print(f"Training samples: {m_train}")
print(f"Test samples: {m_test}")
print(f"X_train shape: {X_train_norm.shape}")
print(f"y_train shape: {y_train.shape}")

## 4 - Building the Model

### One-vs-Rest Strategy

For multi-class classification, we train 3 binary classifiers (one per class) and select the class with highest probability.

### Helper Functions

In [ ]:
def sigmoid(z):
    """
    Compute sigmoid activation
    """
    s = 1 / (1 + np.exp(-z))
    return s

In [ ]:
def initialize_with_zeros(dim):
    """
    Initialize weights with zeros
    """
    w = np.zeros((dim, 1))
    b = 0.0
    return w, b

### Forward and Backward Propagation

In [ ]:
def propagate(w, b, X, Y):
    """
    Compute cost and gradients
    """
    m = X.shape[1]
    
    # Forward propagation
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -(1/m) * np.sum(Y * np.log(A + 1e-8) + (1 - Y) * np.log(1 - A + 1e-8))
    
    # Backward propagation
    dw = (1/m) * np.dot(X, (A - Y).T)
    db = (1/m) * np.sum(A - Y)
    
    cost = np.squeeze(np.array(cost))
    grads = {"dw": dw, "db": db}
    
    return grads, cost

### Optimization

In [ ]:
def optimize(w, b, X, Y, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Optimize weights using gradient descent
    """
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print(f"Cost after iteration {i}: {cost}")
    
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs

In [ ]:
def predict(w, b, X):
    """
    Predict probabilities
    """
    m = X.shape[1]
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    return A

### Complete Model (One-vs-Rest)

In [ ]:
def model_multiclass(X_train, y_train, X_test, y_test, num_classes=3, 
                     num_iterations=1000, learning_rate=0.5, print_cost=False):
    """
    Train one-vs-rest classifiers for multi-class classification
    """
    dim = X_train.shape[0]
    models = []
    
    # Train one classifier per class
    for c in range(num_classes):
        print(f"\n--- Training classifier for class {c} ---")
        
        # Create binary labels (1 if class c, 0 otherwise)
        y_binary = (y_train == c).astype(float)
        
        # Initialize and optimize
        w, b = initialize_with_zeros(dim)
        params, grads, costs = optimize(w, b, X_train, y_binary, 
                                       num_iterations, learning_rate, print_cost)
        
        models.append({"w": params["w"], "b": params["b"], "costs": costs})
    
    # Predict on train and test sets
    train_probs = np.zeros((num_classes, X_train.shape[1]))
    test_probs = np.zeros((num_classes, X_test.shape[1]))
    
    for c in range(num_classes):
        train_probs[c, :] = predict(models[c]["w"], models[c]["b"], X_train)
        test_probs[c, :] = predict(models[c]["w"], models[c]["b"], X_test)
    
    # Select class with highest probability
    y_pred_train = np.argmax(train_probs, axis=0).reshape(1, -1)
    y_pred_test = np.argmax(test_probs, axis=0).reshape(1, -1)
    
    # Calculate accuracy
    train_acc = 100 * np.mean(y_pred_train == y_train)
    test_acc = 100 * np.mean(y_pred_test == y_test)
    
    print(f"\nTraining accuracy: {train_acc:.2f}%")
    print(f"Test accuracy: {test_acc:.2f}%")
    
    return {
        "models": models,
        "y_pred_train": y_pred_train,
        "y_pred_test": y_pred_test,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc
    }

## 5 - Train the Model

In [ ]:
result = model_multiclass(X_train_norm, y_train, X_test_norm, y_test,
                         num_classes=3, num_iterations=1000, 
                         learning_rate=0.5, print_cost=True)

## 6 - Results Analysis

### Learning Curves

In [ ]:
# Plot learning curves for each classifier
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for c in range(3):
    costs = result['models'][c]['costs']
    axes[c].plot(costs)
    axes[c].set_title(f'Class {c} Learning Curve')
    axes[c].set_xlabel('Iterations (x100)')
    axes[c].set_ylabel('Cost')
    axes[c].grid(True)

plt.tight_layout()
plt.show()

### Confusion Matrix

Shows how well the model distinguishes between production levels.

In [ ]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix (ensure all 3 classes are represented)
cm = confusion_matrix(y_test.flatten(), result['y_pred_test'].flatten(), labels=[0, 1, 2])

# Plot using matplotlib
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')

# Add colorbar
plt.colorbar(im, ax=ax)

# Set ticks and labels
class_labels = ['Low', 'Medium', 'High']
ax.set_xticks(np.arange(len(class_labels)))
ax.set_yticks(np.arange(len(class_labels)))
ax.set_xticklabels(class_labels)
ax.set_yticklabels(class_labels)

# Add text annotations
for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        text = ax.text(j, i, cm[i, j], ha="center", va="center", color="black")

ax.set_title('Confusion Matrix - Test Set')
ax.set_ylabel('True Class')
ax.set_xlabel('Predicted Class')
plt.show()

print("\nConfusion Matrix:")
print(cm)

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test.flatten(), result['y_pred_test'].flatten(),
                          target_names=['Low', 'Medium', 'High'],
                          labels=[0, 1, 2],
                          zero_division=0))